In [5]:
# Import necessary modules
# glob allows unix style pathname pattern expansion
from glob import glob
# pydicom is the python dicom reader
import pydicom as dicom
import nibabel.nicom.csareader as csareader
import os
import pandas as pd

In [6]:
# 5-T1w_MPR_vNav -- MPRAGE structrual dicoms
# 6-fMRI_DistortionMap_PA -- fmri field map dicoms
# 7-fMRI_DistortionMap_AP -- fmri field map dicoms opposite phase encode direction
# 9-fMRI_REVL_ROI_loc_2 -- fmri localizer dicoms run1
# 10-fMRI_REVL_Study_1 -- fmri task dicoms run1
# 14-dMRI_DistortionMap_AP_dMRI_REVL -- dwi field map dicoms
# 16-dMRI_AP_REVL -- diffusion weighted dicoms

# directories will have to be specific to your computer
base_dir = '/Users/kcroo/Documents/HW/PSB6351'
subj_dir = '/Mattfeld_REVL-000-vCAT-021-S1/scans/10-fMRI_REVL_Study_1/resources/DICOM/files'

# with strings I can concatenate them with simple addition lines
data_dir = base_dir + subj_dir

# Here I use glob to grab the dicom files
# Why would I use glob?  What does it give me?
# Kate says: glob ensures that we get all files based on pathnames matching a particular requirement
# and 'globs' them together; this can be incredibly specific or far more general - in our case, anything
# that exists in our data directory
data_files = glob(data_dir + '/*')


In [11]:
# Here I am using the python dicom reader to read in a dicom header
# What should go between the square brackets []
# Kate says: A number within the data files array, in order to look at a single subject's functional scan
# in one specific run; While you *can* be specific in terms of which you want, 
# in our current case is just needs to be within the array (so that it gives us a scan)
# So while [0] and [4] and [200] are different scans, in our case they give a similar enough output
data_set = dicom.dcmread(data_files[0])

In [9]:
data_set

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 204
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: MR Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.3.12.2.1107.5.2.43.166003.2019061915475159058436273
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.40.0.13.1.1
(0002, 0013) Implementation Version Name         SH: 'dcm4che-2.0'
(0002, 0016) Source Application Entity Title     AE: 'AN_MEDCOMNT204'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'M', 'ND', 'MOSAIC']
(0008, 0012) Instance Creation Date              DA: '20190619'
(0008, 0013) Instance Creation Time              TM: '154835.377000'
(0008, 0016) SOP Cl

In [13]:
# Elements we're interested in
# Repetition Time
# Echo Time
# Acquisition Matrix 
# Flip Angle
# Acquisition Number
# Protocol Name
# Slice times = data_set[0x00191029].value

print('The number of TRs collected equals: {0}'.format(len(data_files)))
print('The timing of the TR was: {0} ms'.format(data_set.RepetitionTime))
print('The timing of the TE was: {0} ms'.format(data_set.EchoTime))
print('The flip angle in degrees was: {0}'.format(data_set.FlipAngle))
print('The slice thickness was: {0}'.format(data_set.SliceThickness))
print('The name of the scan was: {0}'.format(data_set.ProtocolName))
print('The slice timing was: {0}'.format(data_set[0x00191029].value))
print('The field of view - FOV was: {0}'.format(data_set[int('0051100c', 16)].value))

# Why use the following format to access information in the header?
# print(data_set[int('0051100c', 16)].value) / data_set[0x0051100c].value
# Allows us to access information that is private in the dicom header.

The number of TRs collected equals: 355
The timing of the TR was: 1760 ms
The timing of the TE was: 35 ms
The flip angle in degrees was: 52
The slice thickness was: 2
The name of the scan was: fMRI_REVL_Study_1
The phase encode direction was: COL
COL = AP or PA; ROW = RL or LR
The slice timing was: [1264.99999998, 0.0, 870.0, 77.49999998, 947.49999998, 157.5, 1027.5, 237.49999999, 1107.49999998, 315.0, 1185.0, 475.00000001, 1345.0, 552.49999999, 1422.49999999, 632.49999998, 1502.5, 709.99999999, 1579.99999999, 789.99999998, 1660.0, 394.99999999, 1264.99999998, 0.0, 870.0, 77.49999998, 947.49999998, 157.5, 1027.5, 237.49999999, 1107.49999998, 315.0, 1185.0, 475.00000001, 1345.0, 552.49999999, 1422.49999999, 632.49999998, 1502.5, 709.99999999, 1579.99999999, 789.99999998, 1660.0, 394.99999999, 1264.99999998, 0.0, 870.0, 77.49999998, 947.49999998, 157.5, 1027.5, 237.49999999, 1107.49999998, 315.0, 1185.0, 475.00000001, 1345.0, 552.49999999, 1422.49999999, 632.49999998, 1502.5, 709.9999999

In [15]:
# How do I find phase encoding direction information?
# Kate says: Look through your meta-data! In-plane phase encoding direction is (0018, 1312)
# In addition, finding the "PhaseEncodingDirectionPositive" in the CSA header info gives us additional information

csa_str = data_set[int('00291010', 16)].value
csa_tr = csareader.read(csa_str)
print('The in-plane phase encode direction was: {0}'.format(data_set[int('00181312', 16)].value))
print('COL = AP or PA; ROW = RL or LR')
print('The phase encode direction was: {0}'.format(csa_tr['tags']['PhaseEncodingDirectionPositive']['items'][0]))
print('1 = -; 0 = +')
print('COL, 1 = PA; COL, 0 = AP')

The phase encode direction was: COL
COL = AP or PA; ROW = RL or LR
The phase encode direction was: 0
1 = -; 0 = +
COL, 1 = PA; COL, 0 = AP
